# Face Area Calculations

Computing and working with face areas is important for the analysis of unstructured grids, with many algorithms and workflows requiring them. This section will showcase the different face area calculation options provided with UXarray:

1. Calculate Total Face Area
2. Options for `Grid.calculate_total_face_area` Function
3. Getting Area of Individual Faces
4. Calculate Area of a Single Triangle in Cartesian Coordinates
5. Calculate Area from Multiple Faces in Spherical Coordinates
6. Demonstrate Area Correction at Line of Constant Lattitude

In [ ]:
import uxarray as ux
import numpy as np

We will be using the `outCSne30.ug` grid file, which is encoded in the UGRID convention.

In [ ]:
base_path = "../../test/meshfiles/"
grid_path = base_path + "/ugrid/outCSne30/outCSne30.ug"

ugrid = ux.open_grid(grid_path)
ugrid

## 1. Calculate Total Face Area
We can calculate the total face area by calling the function `Grid.calculate_total_face_area()`. Since our dataset lies on the unit sphere, our expected area is 4π, which is approximately 12.56

In [ ]:
t4_area = ugrid.calculate_total_face_area()
t4_area

## 2. Options for `Grid.calculate_total_face_area` Function

By default, `Grid.calculate_total_face_area` uses a Triangular Quadrature Rule and an Order of 4. However, you can specify the Quadrature Rule and Order as follows:


**Order:**

       1 to 10              for gaussian

       1, 4, 8, 10 and 12   for triangular


In [ ]:
t1_area = ugrid.calculate_total_face_area(quadrature_rule="triangular", order=1)
t1_area

For the result above, notice that the area is slightly different than the first calculation we made.

Now we use Triangular Quadrature Rule and Order 1

Using a lower order is faster, but at the sacrifice of accuracy.

Generally, gaussian quadrature rule is more accurate than the triangular quadrature rule. Additionally, a higher order comes at the cost of computation time, but produces a more accurate result. See `uxarray/grid/area.py` file and function `get_gauss_quadratureDG` for details on quadrature points and weights.

## 3. Getting Area of Individual Faces

We can access the Grid attribute `Grid.face_area` to access the individual face areas. If you have not run a face area calculation yet, it will run the `Grid.compute_face_areas` and cache the value.



In [ ]:
ugrid.face_areas

Now calculate the area again with the `Grid.compute_face_areas` function using arguments: quadrature_rule "gaussian" and order 4

In [ ]:
all_face_areas, all_face_jacobians = ugrid.compute_face_areas(
    quadrature_rule="gaussian", order=4
)
g4_area = all_face_areas.sum()
g4_area

Now we compare the values with actual know value and report error for each of the three cases above.

In [ ]:
actual_area = 4 * np.pi
diff_t4_area = np.abs(t4_area - actual_area)
diff_t1_area = np.abs(t1_area - actual_area)
diff_g4_area = np.abs(g4_area - actual_area)

diff_t1_area, diff_t4_area, diff_g4_area

As we can see, it is clear that the Gaussian Quadrature Rule with Order 4 is the most accurate, and the Triangular Quadrature Rule with Order 1 is the least accurate.


## 4. Calculate Area of a Single Triangle in Cartesian Coordinates

For this section, we create a single triangle face with 3 vertices. By default, in `uxarray`, we assume that the coordinate system is spherical (lat / lon), however if you want to use cartesian coordinates, you must pass through `latlon = False` into the `Grid` constructor.

Assume the units in meters - this is a big triangle!

In [ ]:
verts = [
    [
        [0.0303, -0.7443, 0.6670],
        [-0.3183, -0.7712, 0.5510],
        [0.2055, -0.7162, 0.6670],
    ]
]
# load our vertices into a UXarray Grid object
vgrid = ux.open_grid(verts, latlon=False)

vgrid

In [ ]:
vgrid.calculate_total_face_area()

Additionally, if you are using a unit other than meters, you can update the units as follows

In [ ]:
# Calculate the area of the triangle
area_gaussian = vgrid.calculate_total_face_area(quadrature_rule="gaussian", order=5)
area_gaussian

## 5. Calculate Area from Multiple Faces in Spherical Coordinates

Similar to above, we can construct a `Grid` object with multiple faces by passing through a set of vertices. Here we define 3 six-sided faces.

In [ ]:
faces_verts_ndarray = np.array(
    [
        np.array(
            [
                [150, 10, 0],
                [160, 20, 0],
                [150, 30, 0],
                [135, 30, 0],
                [125, 20, 0],
                [135, 10, 0],
            ]
        ),
        np.array(
            [
                [125, 20, 0],
                [135, 30, 0],
                [125, 60, 0],
                [110, 60, 0],
                [100, 30, 0],
                [105, 20, 0],
            ]
        ),
        np.array(
            [
                [95, 10, 0],
                [105, 20, 0],
                [100, 30, 0],
                [85, 30, 0],
                [75, 20, 0],
                [85, 10, 0],
            ]
        ),
    ]
)

We want our units to be spherical, so we pass through `latlon=True`. Additionally, if `latlon` is not passed through, it will default to spherical coordinates.

In [ ]:
verts_grid = ux.open_grid(faces_verts_ndarray, latlon=True)

verts_grid

In [ ]:
area, jacobian = verts_grid.compute_face_areas()
area

## 6. Area Correction

The correction, \(A\), is calculated using the following formula:

{math}`A = 2\arctan\left(\frac{z(x_{1}y_{2} - x_{2} y_{1})}{x_{1}^2 + y_{1}^2 + x_{1} x_{2} + y_{1} y_{2}} - \frac{z (x_{1} y_{2} - x_{2} y_{1})}{x_{1} x_{2} + y_{1} y_{2}}\right)`

### Where:
<ul>

<li>(x<sub>1</sub>, y<sub>1</sub>, z) are the Cartesian coordinates of the first node.</li>

<li>(x<sub>2</sub>, y<sub>2</sub>, z) are the Cartesian coordinates of the second node (note that the z coordinate is the same for both nodes).</li>

</ul>

### Assumptions:
- This formula assumes that the input coordinates \((x_1, y_1)\) and \((x_2, y_2)\) are normalized (i.e., they lie on the unit sphere).


For the same large triangle used in **Section 4**, we calculate the area correction term when an edge lies along the line of constant latitude.

### The following code:
- Creates a spherical rectangle and calculates the exact area using the spherical excess formula.
- Plots the rectangle, it has two lines of constant lattitude.
- A mesh is formed with uxarray and area is calculated
- Area is again calculated with `correct_area` flag set to `True`

In [ ]:
def theoretical_spherical_rectangle_area(lons, lats, radius=1):
    """
    Calculate the area of a spherical rectangle on a sphere.

    Parameters:
        lons (np.ndarray): Longitudes of the rectangle's corners (in degrees).
        lats (np.ndarray): Latitudes of the rectangle's corners (in degrees).
        radius (float): Radius of the sphere (default is unit sphere).

    Returns:
        float: Area of the spherical rectangle in steradians (or square units if radius is provided).
    """
    # Convert degrees to radians
    lons_rad = np.radians(lons)
    lats_rad = np.radians(lats)

    # Compute longitude and latitude differences
    delta_lambda = abs(lons_rad[0] - lons_rad[2])  # Assuming rectangular shape
    sin_phi_diff = abs(np.sin(lats_rad[0]) - np.sin(lats_rad[2]))

    # Calculate area
    area = radius**2 * delta_lambda * sin_phi_diff
    return area


# Define nodes
node_lon = np.array([90.0, 90.0, 10.0, 10.0])  # Longitudes in degrees
node_lat = np.array([80.0, 10.0, 10.0, 80.0])  # Latitudes in degrees

# Calculate the area
area_steradians = theoretical_spherical_rectangle_area(node_lon, node_lat)
print(
    f"The area of the spherical rectangle is approximately {area_steradians:.6f} steradians"
)

# If Earth's radius is used (R ≈ 6371 km):
earth_radius_km = 6371
area_km2 = theoretical_spherical_rectangle_area(
    node_lon, node_lat, radius=earth_radius_km
)
print(
    f"The area of the spherical rectangle is approximately {area_km2:.2f} square kilometers"
)

In [ ]:
# Define face-node connectivity and create/plot a uxarray object face
face_node_connectivity = np.array([[0, 1, 2, 3]])

face = ux.Grid.from_topology(
    node_lon=node_lon,
    node_lat=node_lat,
    face_node_connectivity=face_node_connectivity,
    fill_value=-1,
)
face.plot(backend="bokeh")

In [ ]:
area_no_correction = face.compute_face_areas(quadrature_rule="gaussian", order=8)

In [ ]:
corrected_area = face.compute_face_areas(
    quadrature_rule="gaussian", order=8, correct_area=True
)

In [ ]:
# Calculate the percentage difference between the corrected and uncorrected areas
percentage_difference = (
    (corrected_area[0][0] - area_no_correction[0][0]) / area_no_correction[0][0] * 100
)

print(
    f"The percentage difference between the corrected and uncorrected areas is {percentage_difference:.2f}%"
)